In [27]:
# import lucem_illud #pip install -U git+git://github.com/Computational-Content-Analysis-2018/lucem_illud.git
import gensim
import requests
import nltk #For stop words and stemmers
from nltk.corpus import stopwords
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn
import sklearn.metrics.pairwise
import sklearn.manifold
import sklearn.decomposition
from os.path import expanduser, join, isfile
from os import listdir
import re
import gzip
import pickle

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 


import os #For looking through files
import os.path #For managing file paths

import article_analysis.parse as aap
import enchant

%load_ext autoreload
%autoreload 1
%aimport article_analysis.parse
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
df = pd.read_json('json_amj.txt', lines=True)
df.shape

(5694, 13)

In [3]:
df.head()

,authors,data,discipline,doi,journaltitle,number,pagecount,pagerange,publication-date,reviewed-work,title,type,volume
0,John B. Miner,{'ocr': ['Communications Exodus-The study also...,"[Business & Economics Collection, Management &...",10.2307/254807,The Academy of Management Journal,1,2,127-128,1972-03-01T00:00:00Z,NaN,Success in Management Consulting and the Conce...,research-article,15
1,Charles R. McClure,{'ocr': ['The author's own recommendations are...,"[Business & Economics Collection, Management &...",10.2307/257446,The Academy of Management Review,2,3,312-314,1980-04-01T00:00:00Z,<product><source>\n <source>Information Strate...,,book-review,5
2,NaN,{'ocr': ['The Academy of Management Learning t...,"[Business & Economics Collection, Management &...",10.2307/3069361,The Academy of Management Journal,3,8,449-456,2001-06-01T00:00:00Z,NaN,Front Matter,misc,44
3,Mark S. Mizruchi,{'ocr': ['Acta Sociologica (1994) 37:329-343 S...,"[Sociology, Social Sciences]",10.2307/4200918,Acta Sociologica,4,15,329-343,1994-01-01T00:00:00Z,NaN,Social Network Analysis: Recent Achievements a...,research-article,37
4,"[Masoud Yasai-Ardekani, Peter H. Grinyer]",{'ocr': ['? Academy of Management Journal 1981...,"[Business & Economics Collection, Management &...",10.2307/255569,The Academy of Management Journal,3,16,471-486,1981-09-01T00:00:00Z,NaN,"Strategy, Structure, Size and Bureaucracy",research-article,24


In [4]:
# df['doi'].to_csv('/Users/belikov/data/jstor/doi.csv.gz', compression='gzip')

In [5]:
# sample = df.loc[200, 'data']
corpus = dict(zip(df['doi'], [x['ocr'] for x in df['data']]))

In [6]:
len(corpus)

5694

In [28]:
fpath = expanduser('~/data/jstor/ngrams_new')
destpath = fpath
suffix = 'pgz'
prefix = 'ngrams_corpus'

In [66]:
def get_present_keys(fpath, prefix, suffix):
    suffix_len = len(suffix)
    prefix_len = len(prefix)
    files = [f for f in listdir(fpath) if isfile(join(fpath, f)) and
             (f[-suffix_len:] == suffix and f[:prefix_len] == prefix)]
    ints = [int(f.split('_')[-1].split('.')[0]) for f in files]
    if ints:
        max_index = max(ints)
    else:
        max_index = 0
    present_keys = set()
    for f in files:
        with gzip.open(join(fpath, f)) as fp:
            item = pickle.load(fp)
            present_keys |= set(list(item.keys()))
    return present_keys, max_index


def split_corpus(corpus, chunk_size=200):
    corpus_keys = list(corpus.keys())
    keys_chunks = [corpus_keys[k:k+chunk_size] for k in range(0, len(corpus_keys), chunk_size)]
    corpus_chunks = [{k: corpus[k] for k in chunk} for chunk in keys_chunks]
    return corpus_chunks

In [69]:
print('len corpus', len(corpus))
ss, max_index = get_present_keys(fpath, prefix, suffix)
keys_to_process = set(corpus.keys()) - ss
print('keys to process len', len(keys_to_process))
remaining_corpus = {k: corpus[k] for k in keys_to_process}
print('remaning corpus len', len(remaining_corpus))
corpus_split = split_corpus(remaining_corpus, 200)

len corpus 5694
keys to process len 0
remaning corpus len 0


In [68]:
for j, chunk in zip(range(len(corpus_split)), corpus_split):
    corpus_ngrams = {}
    for doi, article in chunk.items():
        print('{0} '.format(doi), end='')
        article_phrases = aap.transform_article(article)
        print('{1} '.format(doi, len(article_phrases)), end='')
        ngram_dict = aap.compute_ngrams(article_phrases)
        corpus_ngrams[doi] = ngram_dict
    with gzip.open(join(destpath, 'ngrams_corpus_{0}.pgz'.format(j + 1+ max_index )), 'wb') as fp:
        pickle.dump(corpus_ngrams, fp)
    print('\n{0} chunks, {1:.2f} % complete'.format(j, 100*(j)/len(corpus_split)))

10.2307/256340 5 10.2307/256625 428 10.2307/258465 787 10.2307/4134356 383 10.2307/259039 596 10.2307/255306 128 10.2307/684610 517 10.2307/255654 99 10.2307/3069284 628 10.2307/255529 268 10.2307/258530 180 10.2307/256378 538 10.2307/255782 109 10.2307/255288 249 10.2307/2393201 843 10.2307/1556373 649 10.2307/257781 209 10.2307/2393616 801 10.2307/23070653 301 10.2307/258647 511 10.2307/20159260 452 10.2307/256705 419 10.2307/256305 590 10.2307/2580414 367 10.2307/255428 316 10.2307/254756 222 10.2307/3069419 698 10.2307/258824 863 10.2307/257302 268 10.2307/20159631 336 10.2307/20159609 757 10.2307/255631 182 10.2307/30037209 1008 10.2307/3069325 884 10.2307/3069310 407 10.2307/3598350 493 10.2307/258555 625 10.2307/254935 74 10.2307/2393108 392 10.2307/256645 798 10.2307/20159735 398 10.2307/20159485 688 10.2307/256990 348 10.2307/256516 282 10.2307/30040638 585 10.2307/20159911 93 10.2307/2393315 692 10.2307/256654 679 10.2307/255859 582 10.2307/254948 371 10.2307/20159572 499 10.

10.2307/256013 257 10.2307/20159795 814 10.2307/1556397 725 10.2307/2393357 60 10.2307/1556334 885 10.2307/254804 199 10.2307/41940764 536 10.2307/256817 750 10.2307/258214 594 10.2307/20159917 489 10.2307/20159687 540 10.2307/20159874 976 10.2307/254682 70 10.2307/254674 256 10.2307/258788 1125 10.2307/255005 181 10.2307/257700 541 10.2307/255614 181 10.2307/255883 206 10.2307/258092 605 10.2307/256241 348 10.2307/20159845 1119 10.2307/2393267 85 10.2307/256155 280 10.2307/30040613 583 10.2307/2667113 66 10.2307/2391853 366 10.2307/256296 414 10.2307/256317 583 10.2307/2392097 507 10.2307/256211 195 10.2307/255057 122 10.2307/30040642 375 10.2307/256845 565 10.2307/20159919 922 10.2307/20159663 237 10.2307/257043 80 10.2307/255739 416 10.2307/255661 275 10.2307/254829 191 10.2307/256501 726 10.2307/30040643 428 10.2307/256974 445 10.2307/41940761 536 10.2307/255674 293 10.2307/1556356 1122 10.2307/255361 274 10.2307/255487 120 10.2307/27749301 304 10.2307/30040651 673 10.2307/20159496

10.2307/1556388 408 10.2307/20159907 317 10.2307/20159705 289 10.2307/256044 428 10.2307/1171154 244 10.2307/255813 455 10.2307/20159648 448 10.2307/256853 327 10.2307/20159756 57 10.2307/257252 385 10.2307/20159554 266 10.2307/257176 413 10.2307/255149 49 10.2307/254694 260 10.2307/256116 338 10.2307/255211 21 10.2307/254703 152 10.2307/256992 459 10.2307/20159873 845 10.2307/257959 309 10.2307/256742 620 10.2307/255948 282 10.2307/23479121 420 10.2307/1389371 333 10.2307/257723 351 10.2307/258980 657 10.2307/259212 438 10.2307/41307830 550 10.2307/256088 180 10.2307/2393929 71 10.2307/20159237 812 10.2307/254868 79 10.2307/259161 705 10.2307/254887 140 10.2307/258039 537 10.2307/259103 937 10.2307/41721953 949 10.2307/20159344 749 10.2307/256671 1198 10.2307/257270 259 10.2307/3090109 463 10.2307/40645825 787 10.2307/20159011 562 10.2307/256454 64 10.2307/258493 330 10.2307/258400 495 10.2307/2580166 520 10.2307/2748727 218 10.2307/256633 788 10.2307/254997 44 10.2307/255046 211 10.2

10.2307/254977 149 10.2307/254895 50 10.2307/255464 322 10.2307/2393565 439 10.2307/254899 282 10.2307/255979 284 10.2307/3069291 549 10.2307/255018 14 10.2307/41307832 503 10.2307/256888 540 10.2307/20159550 764 10.2307/255499 329 10.2307/255720 356 10.2307/256051 235 10.2307/20159903 40 10.2307/254841 70 10.2307/3090204 574 10.2307/255176 179 10.2307/2393593 591 10.2307/257192 288 10.2307/23027556 551 10.2307/255577 268 10.2307/257135 416 10.2307/254964 201 10.2307/27928070 656 10.2307/255886 184 10.2307/256962 599 10.2307/20159246 74 10.2307/254910 265 10.2307/4134367 1094 10.2307/255605 311 10.2307/258216 399 10.2307/27522155 504 10.2307/255061 276 10.2307/255433 319 10.2307/2393494 759 10.2307/255087 59 10.2307/20159589 417 10.2307/1046097 286 10.2307/2393172 619 10.2307/20159846 625 10.2307/256959 219 10.2307/256836 376 10.2307/256464 329 10.2307/257696 438 10.2307/3069365 761 10.2307/255120 351 10.2307/256747 399 10.2307/2393124 440 10.2307/2393449 1128 10.2307/2989896 648 10.23

10.2307/255163 95 10.2307/255791 476 10.2307/20159595 166 10.2307/1556361 638 10.2307/4121457 490 10.2307/255524 119 10.2307/161653 554 10.2307/255284 228 10.2307/258734 182 10.2307/20159207 918 10.2307/256937 195 10.2307/255561 206 10.2307/255704 441 10.2307/258607 1206 10.2307/20109954 823 10.2307/257134 197 10.2307/255898 377 10.2307/255420 156 10.2307/259102 970 10.2307/2782073 624 10.2307/255241 124 10.2307/254766 20 10.2307/20159357 796 10.2307/20159752 545 10.2307/41012146 334 10.2307/254748 230 10.2307/2393080 542 10.2307/255910 426 10.2307/256917 629 10.2307/257698 300 10.2307/3069358 378 10.2307/254891 130 10.2307/20159385 835 10.2307/255533 216 10.2307/255080 196 10.2307/255624 161 10.2307/255904 504 10.2307/254974 272 10.2307/255247 42 10.2307/2392667 536 10.2307/3069295 524 10.2307/1556365 343 10.2307/255192 18 10.2307/256532 403 10.2307/257855 338 10.2307/25472538 630 10.2307/258634 535 10.2307/2787140 559 10.2307/256007 762 10.1525/sp.2004.51.4.505 639 10.2307/255128 305

...

10.2307/254874 336 10.2307/254812 295 10.2307/2393737 759 10.2307/258917 637 10.2307/353273 347 10.2307/2393766 33 10.2307/257760 364 10.2307/255663 297 10.2307/3094808 916 10.2307/256163 301 10.2307/254914 80 10.2307/1556423 357 10.2307/1556384 1054 10.2307/255879 458 10.2307/1556374 658 10.2307/2392855 470 10.2307/20159839 353 10.2307/27526956 757 10.2307/256135 548 10.2307/2393935 634 10.2307/255641 369 10.2307/256278 389 10.2307/256523 377 10.2307/2391745 278 10.2307/254654 301 10.2307/256141 189 10.2307/3069363 35 10.2307/2392715 243 10.2307/256444 643 10.2307/256843 755 10.2307/255659 224 10.2307/255632 198 10.2307/258477 224 10.2307/2393989 412 10.2307/259377 667 10.2307/27801510 632 10.2307/254986 109 10.2307/255435 295 10.2307/254702 100 10.2307/2392883 697 10.2307/255223 35 10.2307/20159784 597 10.2307/255230 3 10.2307/40184654 311 10.2307/254807 33 10.2307/256004 167 10.2307/2580351 686 10.2307/258867 883 10.2307/257929 295 10.2307/255346 151 10.2307/256702 802 10.2307/30696

10.2307/257360 420 10.2307/257602 392 10.2307/30040679 286 10.2307/3069318 1093 10.2307/255443 294 10.2307/2392287 330 10.2307/254720 61 10.2307/259209 1171 10.2307/4148783 457 10.2307/256721 548 10.2307/257245 430 10.2307/256757 750 10.2307/256620 557 10.2307/1556424 262 10.2307/255088 89 10.2307/3512571 416 10.2307/257002 754 10.2307/2393756 907 10.2307/258314 529 10.2307/258090 599 10.2307/255017 89 10.2307/254763 42 10.2307/256927 621 10.2307/2392269 119 10.2307/20159785 98 10.2307/20159264 833 10.2307/3094849 749 10.2307/27749323 1053 10.2307/255220 132 10.2307/41303971 679 10.2307/1556403 699 10.2307/254845 96 10.2307/257084 316 10.2307/258977 806 10.2307/255202 57 10.2307/255953 130 10.2307/255850 143 10.2307/255765 128 10.2307/591789 596 10.2307/255968 163 10.2307/255079 326 10.2307/255985 290 10.2307/255164 25 10.2307/20159333 635 10.2307/257090 578 10.2307/258789 836 10.2307/256023 369 10.2307/255406 130 10.2307/29789589 475 10.2307/256515 382 10.2307/255191 62 10.2307/258029

10.2307/254811 43 10.2307/2393314 537 10.2307/255995 419 10.2307/256865 545 10.2307/3069306 749 10.2307/255353 190 10.2307/254886 154 10.2307/255135 182 10.2307/256964 680 10.2307/2667085 603 10.2307/2393810 669 10.2307/257025 397 10.2307/259223 800 10.2307/20159804 9 10.2307/255621 379 10.2307/1556414 539 10.2307/3556659 1414 10.2307/256861 548 10.2307/27809266 455 10.2307/255518 191 10.2307/20159313 1088 10.2307/256590 529 10.2307/256855 199 10.2307/27928200 591 10.2307/256047 307 10.2307/256100 276 10.2307/258109 641 10.2307/254689 300 10.2307/256869 155 10.2307/259092 168 10.2307/258608 750 10.2307/256176 268 10.2307/2393698 696 10.2307/255109 84 10.2307/256778 375 10.2307/254663 100 10.2307/2391958 281 10.2307/255381 588 10.2307/2393111 57 10.2307/257330 611 10.2307/257000 630 10.2307/255831 241 10.2307/3069347 178 10.2307/4121512 352 10.2307/257061 405 10.2307/20159505 136 10.2307/258041 499 10.2307/20159739 246 10.2307/254833 171 10.2307/255992 155 10.2307/256160 305 10.2307/201

10.2307/256467 197 10.2307/254765 110 10.2307/258572 338 10.2307/40645818 807 10.2307/255667 257 10.2307/190405 522 10.2307/255270 269 10.2307/20159680 442 10.2307/2392364 215 10.2307/3069353 371 10.2307/20159794 516 10.2307/256975 555 10.2307/255339 15 10.2307/4134387 787 10.2307/30040630 414 10.2307/23325436 422 10.2307/257008 758 10.2307/1048009 137 10.2307/23160428 559 10.2307/1556346 519 10.2307/2393610 96 10.2307/41682638 355 10.2307/255229 222 10.2307/257446 109 10.2307/256918 578 10.2307/270931 693 10.2307/255877 288 10.2307/30040673 599 10.2307/258178 371 10.2307/256973 716 10.2307/257701 336 10.2307/255965 232 10.2307/255816 124 10.2307/256833 763 10.2307/3593088 585 10.2307/255902 334 10.2307/3069425 303 10.2307/255042 237 10.2307/1556385 692 10.2307/257222 372 10.2307/2776471 357 10.2307/257009 638 10.2307/2667070 939 10.2307/20159279 996 10.2307/259288 1005 10.2307/256955 385 10.2307/257047 83 10.2307/256977 518 10.2307/255216 330 10.2307/20159923 302 10.2307/255020 185 10

10.2307/259270 720 10.2307/20159893 580 10.2307/254967 237 10.2307/254907 21 10.2307/20109929 1064 10.2307/255078 72 10.2307/27749353 237 10.2307/257989 221 10.2307/258305 166 10.2307/256260 222 10.2307/20159731 641 10.2307/3094826 752 10.2307/42629854 752 10.2307/256245 194 10.2307/3069331 152 10.2307/20159895 737 10.2307/254692 269 10.2307/256710 210 10.2307/3322450 510 10.2307/254677 17 10.2307/20159870 154 10.2307/2393615 750 10.2307/258414 753 10.2307/2393738 565 10.2307/255833 271 10.2307/25054324 991 10.2307/41723081 790 10.2307/257828 277 10.2307/258362 188 10.2307/2392342 518 10.2307/20159792 519 10.2307/20159228 306 10.2307/258291 700 10.2307/255560 246 10.2307/41683001 450 10.2307/2393291 75 10.2307/256841 179 10.2307/258853 631 10.2307/256857 688 10.2307/254863 39 10.2307/4131473 531 10.2307/255764 120 10.2307/256956 579 10.2307/256042 327 10.2307/259162 205 10.2307/257695 530 10.2307/30037220 733 10.2307/20109953 1148 10.2307/41328589 622 10.2307/254799 293 10.2307/256194 

In [12]:
# doi_ = '10.2307/3069361'
# doi_ = '10.2307/254807'
# doi_ = '10.2307/2667048'
# doi_ = '10.2307/254961'
# doi_ = '10.1525/sop.2008.51.1.163'
# mask = (df['doi'] == doi_)
# article = df.loc[mask, 'data'].values[0]['ocr']
# # ar_phrases = aap.transform_article(article, verbose=True)
# ar_phrases = aap.transform_article(article)
# ng_dict = aap.compute_ngrams(ar_phrases)
# ng_dict[2]